## Geoserver rest API
* [API docs](https://docs.geoserver.org/latest/en/user/rest/index.html#rest) BROKEN
* [API legacy](https://docs.geoserver.org/2.13.2/user/rest/api/index.html#rest-api)
* XML :-/
* Useful link: curl to requests at https://curl.trillworks.com/
* Server running at http://localhost:8080/geoserver

First demo a get with CURL following [the basic docs](https://docs.geoserver.org/latest/en/user/rest/api/details.html). Note that `Content-type` can be XML, JSON, ZIP amonst others

In [1]:
!curl -u admin:geoserver -XGET -H "Accept: text/xml" http://localhost:8080/geoserver/rest/about/version.xml

<about>
  <resource name="GeoServer">
    <Build-Timestamp>21-Sep-2018 20:18</Build-Timestamp>
    <Version>2.14.0</Version>
    <Git-Revision>2ecf6413b0d5d304c7dc23d8d7e9f3cfad29f37c</Git-Revision>
  </resource>
  <resource name="GeoTools">
    <Build-Timestamp>21-Sep-2018 16:31</Build-Timestamp>
    <Version>20.0</Version>
    <Git-Revision>ffc271f317c04e714ea44a4879dd4601bd723d5e</Git-Revision>
  </resource>
  <resource name="GeoWebCache">
    <Version>1.14.0</Version>
    <Git-Revision>1.14.x/775e94eb2ae59dd9b74aecc709b67d7b968a4317</Git-Revision>
  </resource>
</about>

Alternatively using json

In [2]:
!curl -u admin:geoserver -XGET -H "Accept: text/json" http://localhost:8080/geoserver/rest/about/version.json

{"about":{"resource":[{"@name":"GeoServer","Build-Timestamp":"21-Sep-2018 20:18","Version":"2.14.0","Git-Revision":"2ecf6413b0d5d304c7dc23d8d7e9f3cfad29f37c"},{"@name":"GeoTools","Build-Timestamp":"21-Sep-2018 16:31","Version":20,"Git-Revision":"ffc271f317c04e714ea44a4879dd4601bd723d5e"},{"@name":"GeoWebCache","Version":"1.14.0","Git-Revision":"1.14.x\/775e94eb2ae59dd9b74aecc709b67d7b968a4317"}]}}

Now use requests

In [3]:
import requests

headers = {
    'Accept': 'text/json',
}


response = requests.get('http://localhost:8080/geoserver/rest/about/version.json', headers=headers, auth=('admin', 'geoserver'))

#NB. Original query string below. It seems impossible to parse and
#reproduce query strings 100% accurately so the one below is given
#in case the reproduced version is not "correct".
# response = requests.get('http://localhost:8080/geoserver/rest/about/manifest.xml?key=GeoServerModule', headers=headers, auth=('admin', 'geoserver'))


In [4]:
print(response.text)

{"about":{"resource":[{"@name":"GeoServer","Build-Timestamp":"21-Sep-2018 20:18","Version":"2.14.0","Git-Revision":"2ecf6413b0d5d304c7dc23d8d7e9f3cfad29f37c"},{"@name":"GeoTools","Build-Timestamp":"21-Sep-2018 16:31","Version":20,"Git-Revision":"ffc271f317c04e714ea44a4879dd4601bd723d5e"},{"@name":"GeoWebCache","Version":"1.14.0","Git-Revision":"1.14.x\/775e94eb2ae59dd9b74aecc709b67d7b968a4317"}]}}


Lets convert this json string to a python dict

In [5]:
import json

In [6]:
data_dict = json.loads(response.text)
data_dict

{'about': {'resource': [{'@name': 'GeoServer',
    'Build-Timestamp': '21-Sep-2018 20:18',
    'Version': '2.14.0',
    'Git-Revision': '2ecf6413b0d5d304c7dc23d8d7e9f3cfad29f37c'},
   {'@name': 'GeoTools',
    'Build-Timestamp': '21-Sep-2018 16:31',
    'Version': 20,
    'Git-Revision': 'ffc271f317c04e714ea44a4879dd4601bd723d5e'},
   {'@name': 'GeoWebCache',
    'Version': '1.14.0',
    'Git-Revision': '1.14.x/775e94eb2ae59dd9b74aecc709b67d7b968a4317'}]}}

Nice, so no need to use [requests-xml](https://github.com/erinxocon/requests-xml/)

In [7]:
[resource['@name'] for resource in data_dict['about']['resource']]

['GeoServer', 'GeoTools', 'GeoWebCache']

# Post to Geoserver
Lets now post some data to Geoserver. The [Geoserver rest examples](https://docs.geoserver.org/latest/en/user/rest/api/index.html) give some advice

Using requests files are posted like:
```python
url = 'https://httpbin.org/post'
files = {'file': open('report.xls', 'rb')}

r = requests.post(url, files=files)
>>> r.text
```